Comprehensive Analysis of the Indian Premier League and Projecting the Optimal Squad for the 2025 Season

In [1]:
# Load Dataset
import pandas as pd
dataset = pd.read_csv("Dataset/IPL_Meta_Data.csv")
df = dataset
metadata = dataset
# dataset.head()

C:\Users\niran\AppData\Local\Temp\ipykernel_19968\3160474569.py:3: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv("Dataset/IPL_Meta_Data.csv")


This dataset contains information about the IPL data, contains ball by ball data from 2008 to 2024. 


In [2]:
# This is the sample data
dataset.head(1)

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,...,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed
0,1426312,2024,5/26/2024,"MA Chidambaram Stadium, Chepauk, Chennai",1,0.1,Sunrisers Hyderabad,Kolkata Knight Riders,Abhishek Sharma,TM Head,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We are cleaning the data and trying to create different tables namely:
Player Table 
Batsman table,
Bowlers table, 
Fielding table, 
matches table

In [3]:
# Column names and Data types 
dataset.dtypes

match_id                    int64
season                     object
start_date                 object
venue                      object
innings                     int64
ball                      float64
batting_team               object
bowling_team               object
striker                    object
non_striker                object
bowler                     object
runs_off_bat                int64
extras                      int64
wides                     float64
noballs                   float64
byes                      float64
legbyes                   float64
penalty                   float64
wicket_type                object
player_dismissed           object
other_wicket_type         float64
other_player_dismissed    float64
dtype: object

1. Players Table: 
table name: player_info

In [86]:
unique_players = pd.concat([df['striker'], df['non_striker'], df['bowler']]).drop_duplicates().sort_values()

player_info = pd.DataFrame(columns=['player', 'seasons'])

player_list = []
seasons_list = []

for player in unique_players:
    player_rows = df[(df['striker'] == player) | (df['non_striker'] == player) | (df['bowler'] == player)]
    seasons = ', '.join(sorted(set(player_rows['season'].astype(str))))
    player_list.append(player)
    seasons_list.append(seasons)

player_info['player'] = player_list
player_info['seasons'] = seasons_list

player_info['seasons'] = player_info['seasons'].apply(lambda x: ', '.join(sorted(set(x.split(', ')), key=x.split(', ').index)))

In [89]:
# Count the number of seasons a player played
player_info['num_seasons'] = player_info['seasons'].apply(lambda x: len(x.split(', ')))


In [91]:
player_info
# player_info.to_csv("dummydata.csv")